In [28]:
import pandas as pd
import numpy as np
import streamlit as st
import openai
import time
from bs4 import BeautifulSoup
import requests
import re
import os
import json


In [52]:

# # load the dataframes
# def load_dataframes():
#     csv_url_path = "./data/Products and Categories.xlsx"
#     tags_url_path = "./data/dummy - Product category and tags.xlsx"
#     df_url = pd.read_excel(csv_url_path)
#     df_tags = pd.read_excel(tags_url_path)
#     return df_url, df_tags


# def process_dataframes(df_url, df_tags):
#     print(len(df_url))
#     print(len(df_tags))

#     # set row to as header
#     print(df_tags.iloc[0])
#     df_tags.columns = df_tags.iloc[0]
#     df_url = df_url.drop_duplicates(subset=['Product Name'])

#     # Merge df_tags and df_url on 'Product_Name', only adding the 'url' column
#     df_tags = df_tags.merge(df_url[["Product Name", "url"]], on="Product Name", how="left")

#     # drop columns nan, Technology, General, Related products, Varient products
#     df_tags = df_tags.drop(
#         columns=[
#             "Technology",
#             "General",
#             "Related products",
#             "Variant products",
#             "Contains / made from products",
#         ],
#         axis=1,
#     )
#     print(len(df_tags))
#     # add the string "https://www.kongsberg.com/" to the url column for each url
#     df_tags["url"] = "https://www.kongsberg.com" + df_tags["url"].astype(str)
#     # rename the column Product Name to Product_Name
#     df_tags = df_tags.rename(columns={"Product Name": "Product_Name"})

#     return df_tags
# df_url, df_tags = load_dataframes()
# df_tags = process_dataframes(df_url, df_tags)

In [93]:
df_tags = pd.read_excel("data/df_tags_use_app_22_11_issoftware.xlsx", usecols=["Product_Name","Product category", "url", "image_url", "is_software"])
print(len(df_tags))
# Split the 'Product category' column into multiple rows
df_tags = df_tags.assign(
    **{"Product category": df_tags["Product category"].str.split(",")}
).explode("Product category")

# Trim whitespace from the 'Product category' column
df_tags["Product category"] = df_tags["Product category"].str.strip()


print(len(df_tags))



215
298


In [94]:
print(df_tags["url"][3])
print(df_tags["Product category"][3])
print(df_tags["Product_Name"][3])


https://www.kongsberg.com/maritime/products/Acoustics-Positioning-and-Communication/transponders/cnode-transponders-for-hipap-and-uPAP/cNODE-MiniS/
Underwater navigation & positioning
cNODE MiniS - Transponder


In [86]:
# def extract_text(url):
#     # Get the HTML of the page
#     response = requests.get(url)
#     html = response.text

#     # Parse the HTML with BeautifulSoup
#     soup = BeautifulSoup(html, 'html.parser')

#     # Find all elements with the specified class
#     elements = soup.find_all(class_="RichtextArea ProductPage__richtext text-wrapper")

#     # Extract the text of each element
#     rich_text = [element.get_text(strip=True) for element in elements]
#     rich_text_string = ' '.join(rich_text)

#     link = soup.find('a', href='#technicalInformation')

#     bullet_points = []
#     bullet_points_string = ""
#     # If the link was found, find the element it links to
#     if link is not None:
#         target_id = link['href'].lstrip('#')
#         target_element = soup.find(id=target_id)

#         # If the target element was found, get its text
#         if target_element is not None:
#             # Find all the list items in the target element
#             list_items = target_element.find_all('li')

#             # Extract the text of each list item
#             bullet_points = [li.get_text(strip=True) for li in list_items]
#             bullet_points_string = '\n'.join(bullet_points)

        
#     text = rich_text_string +" \n "+ bullet_points_string
#     return text
# url_text_dict = {}
# for index, row in df_tags.iterrows():
#     url = row["url"]
#     text = extract_text(url)
#     url_text_dict[url] = text
#     print(index, "/", len(df_tags))


# # save the dict as a json file
# print(url_text_dict)

In [4]:
# load the dict from pickle file
import pickle
with open('./data/url_technical_text_dict.pkl', 'rb') as handle:
    url_text_dict = pickle.load(handle)

In [5]:
print(url_text_dict)

{'https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/multibeam-echo-sounders/k-sync/': "Performing a survey operation with more than one echo sounder presents a challenge. It is well known that running several acoustic systems simultaneously can cause interference between the systems, which may reduce the data quality.Interference between different acoustic devices and instruments can be a severe problem for ships with several acoustic instruments operating simultaneously. The effect of such interference will vary according to the configuration, from slight disturbances to complete malfunctioning. The KONGSBERG synchronizing unit, K-SYNC is designed to solve these problems by proper timing of the instruments and by controlling the triggering of each instrument's transmission.Kongsberg K-Sync works across the range of KONGSBERG products, but also supports echo sounders from other suppliers ensuring synchronization across your entire echo sounder portfolio. The sys

In [89]:
# # save the dict as pickle file
# import pickle
# with open('./data/url_technical_text_dict.pkl', 'wb') as handle:
#     pickle.dump(url_text_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [8]:
from openai._client import OpenAI

client = OpenAI(
    api_key=st.secrets["openai"]["api_key"],
)

In [91]:
# def general_gpt(prompt: str):
#     # make chat gpt completion function with streamlit
#     openai.api_key = st.secrets["openai"]["api_key"]
#     openai.api_type = "azure"
#     openai.api_base = "https://cog-fxpoc-tonality-dev-01.openai.azure.com/"
#     openai.api_version = "2023-03-15-preview"
#     # gpt_model = "gpt-35-turbo"
#     gpt_model = "gpt-35-turbo-16k"
#     # gpt_model = "gpt-4"
#     completion = openai.ChatCompletion.create(
#         deployment_id=gpt_model,
#         messages=[
#             {
#                 "role": "user",
#                 "content": "{}".format(prompt),
#             }
#         ],
#         temperature=0.3,
#         max_tokens=1500,
#         top_p=1.0,
#         frequency_penalty=0.1,
#         presence_penalty=0.1,
#     )
#     return str(completion.choices[0].message.content)

In [101]:
import time

def generate_tags_and_handle_rate_limit(df_tags):
    df_tags["new_tags"] = ""

    # Iterate over the rows of the dataframe
    for index, row in df_tags.iterrows():
        print(index)
        # Create a prompt using the Product_Name, category, and the text from the website
        url = row["url"]
        product_name = row["Product_Name"]
        product_category = row["Product category"]

        website_text = url_text_dict.get(url, "")

        prompt = f"Given the product description: '{website_text}', product category {product_category} and the product name: '{product_name}', make an appropiate number of tags per product, but no more than 6. These tags should be derived from the product description and be applicable to this and similar products. Please avoid using the product name directly as a tag. Your response should be a comma-separated list of the number of tags of your choice."
        # Call the general_gpt function with this prompt

        for _ in range(3):  # Retry up to 3 times
            try:
                response = client.chat.completions.create(
                    # model="gpt-3.5-turbo-1106",
                    model="gpt-4-1106-preview",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that generates relevant tags for products. Your responses should be in JSON format and consist of a comma-separated list of tags. return json with following format: {'tags': ['tag1', 'tag2', 'tag3, etc...']}"},
                        {"role": "user", "content": "{}".format(prompt)}
                    ],
                    max_tokens=200,
                    response_format={ "type": "json_object" },
                    timeout=10  # Add a timeout
                )
                tags = response.choices[0].message.content.strip()
                print(tags)
                # Save the generated tags in the 'new_tags' column
                df_tags.loc[index, "new_tags"] = tags
                break
            except Exception as e:
                print(f"Error at index {index}: {e}")
                time.sleep(5)  # Wait for 5 seconds before retrying
        else:
            print(f"Skipping index {index} after 3 failed attempts")
            continue  # Skip the current index if the API call fails 3 times

    return df_tags

df_tags_tech = generate_tags_and_handle_rate_limit(df_tags)
df_tags_tech.to_csv("./data/tags_27_11.csv", index=False)

0
{"tags": ["echo sounder synchronization", "acoustic interference management", "sonar sequencing", "real-time acoustic synchronization", "underwater navigation", "multibeam echosounder compatibility"]}
1
{
  "tags": ["underwater navigation", "positioning transponder", "marine technology", "acoustic communication", "subsea equipment"]
}
2
{
  "tags": ["underwater navigation", "transducer integration", "SSBL positioning", "LBL positioning", "subsea telemetry", "acoustic positioning"]
}
3
{"tags": ["marine transponder", "subsea positioning", "acoustic telemetry", "deep sea exploration", "rechargeable Li-Ion battery", "tilt sensor"]}
4
{"tags": ["underwater acoustic positioning", "modular transponder design", "SSBL positioning", "LBL positioning", "acoustic data link", "internal tilt sensor"]}
5
{"tags": ["underwater_acoustic_telemetry", "omnidirectional_sonar", "depth_rated_600m", "battery_status_monitoring", "integrated_tilt_sensor", "LBL_SSBL_positioning"]}
6
{
  "tags": [
    "subsea 

In [102]:
import ast

tag_list = []

for row in df_tags_tech.itertuples():
    tags_dict_str = row.new_tags
    try:
        tags_dict = ast.literal_eval(tags_dict_str)
    except (SyntaxError, ValueError):
        print(f"Error parsing string to dict: {tags_dict_str}")
        continue

    for tag_category, tags in tags_dict.items():
        # Assuming tags is a list of strings
        for tag in tags:
            tag_list.append(tag)

print(tag_list)
print(len(tag_list))

['echo sounder synchronization', 'acoustic interference management', 'sonar sequencing', 'real-time acoustic synchronization', 'underwater navigation', 'multibeam echosounder compatibility', 'underwater navigation', 'positioning transponder', 'marine technology', 'acoustic communication', 'subsea equipment', 'underwater navigation', 'transducer integration', 'SSBL positioning', 'LBL positioning', 'subsea telemetry', 'acoustic positioning', 'marine transponder', 'subsea positioning', 'acoustic telemetry', 'deep sea exploration', 'rechargeable Li-Ion battery', 'tilt sensor', 'underwater acoustic positioning', 'modular transponder design', 'SSBL positioning', 'LBL positioning', 'acoustic data link', 'internal tilt sensor', 'underwater_acoustic_telemetry', 'omnidirectional_sonar', 'depth_rated_600m', 'battery_status_monitoring', 'integrated_tilt_sensor', 'LBL_SSBL_positioning', 'subsea monitoring', 'sensor logging', 'acoustic telemetry', 'environmental monitoring', 'data acquisition', 'sub

In [103]:
categories = df_tags_tech["Product category"].dropna().unique()

# Split the categories that have multiple categories with comma
categories = [category.split(",") for category in categories]

# Make the list into without duplicates
categories = list(set([item for sublist in categories for item in sublist]))

categories = [category.strip() for category in categories]


In [104]:
unique_categories = set(categories)

for category in unique_categories:
    print(category)

naval
Fishery & Ocean research
Communication solutions
Surveilance & monitoring
Fish finding
Seafloor mapping
Autonomous and uncrewed solutions
Surface navigation & positioning
Inertial solutions
Underwater navigation & positioning
Cloud solution
Fishery & ocean research
Geophysical survey
Naval


In [109]:
tags = tag_list

# Create a dictionary to store the tags
tags_dict = {}

for category in set(categories):
    # Get the products in the current category
    products = df_tags_tech[df_tags_tech["Product category"] == category]["Product_Name"]
    # Get the tags in the current category
    # Assuming 'df' is your DataFrame and 'category' is the current category
    tags_in_category = df_tags_tech[df_tags_tech['Product category'] == category]['new_tags']

    all_tags_in_category = []
    for tags_dict_str in tags_in_category:
        try:
            tags_dict_temp = ast.literal_eval(tags_dict_str)
            for tags in tags_dict_temp.values():
                all_tags_in_category.extend(tags)
        except (SyntaxError, ValueError):
            print(f"Error parsing string to dict: {tags_dict_str}")

    # Remove duplicates
    all_tags_in_category = list(set(all_tags_in_category))

    messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that generates relevant tags for products. Your responses should be in JSON format and consist of a dictionary where the keys are the top-level tags and the values are lists of the sub-tags."
    },
    {
        "role": "user",
        "content": f"""Given the list of sub-tags: {all_tags_in_category} and the products: {products} in category {category}: Use these top-level tags:
        ['Product Type', 'Technology', 'Application']. Product type is what type of product it is. Technology is what technology the product uses. Application is what the product is used for. The technical should be more technical than the application and prouduct type.
         For each top-level tag, select specific and descriptive sub-tags from the list of sub-tags. Use as many subtags as you think is necessary to describe the products in the category. But no more than 4. Try to have unique sub-tags for each top-level tag. 
         Use tags from the list of sub-tags only once. Do not use the same sub-tag for multiple top-level tags. Choose the sub-level tags that are most relevant for the top-level tag. Do not use Product Names as tags. Only use 4 tags per top-level tag."""
    }
    ]

    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        response_format={ "type": "json_object" }
    )
    category_tags = response.choices[0].message.content.strip()
    print(category_tags)

    # Save the tags in the dictionary
    tags_dict[category] = category_tags
print(tags_dict)

{
  "Product Type": ["Autonomous Underwater Vehicle", "Diver Detection Sonar"],
  "Technology": ["advanced navigation system", "3D sonar technology", "multi-sensor payload", "adaptive autonomy"],
  "Application": ["underwater exploration", "maritime surveillance", "harbor defense", "marine survey"]
}
{
    "Product Type": ["Uncrewed Surface Vehicle"],
    "Technology": ["remote vessel operation", "customizable payload", "USV hydroacoustic applications"],
    "Application": ["long endurance", "maritime research support", "autonomous maritime vehicle"]
}
{
    "Product Type": ["Modem", "Radio", "Software", "Transceiver"],
    "Technology": ["VDES technology", "acoustic modem", "reconfigurable SDR", "sonar software"],
    "Application": ["maritime communication", "subsea communication", "underwater mapping", "navigation systems"]
}
{
    "Product Type": ["Autonomous Underwater Vehicle", "Acoustic Control System", "Leak Detection System", "Diver Detection Sonar"],
    "Technology": ["3D so

In [110]:
print(tags_dict)
import json

with open('tags_dict.json', 'w') as f:
    json.dump(tags_dict, f)

{'naval': '{\n  "Product Type": ["Autonomous Underwater Vehicle", "Diver Detection Sonar"],\n  "Technology": ["advanced navigation system", "3D sonar technology", "multi-sensor payload", "adaptive autonomy"],\n  "Application": ["underwater exploration", "maritime surveillance", "harbor defense", "marine survey"]\n}', 'Fishery & Ocean research': '{\n    "Product Type": ["Uncrewed Surface Vehicle"],\n    "Technology": ["remote vessel operation", "customizable payload", "USV hydroacoustic applications"],\n    "Application": ["long endurance", "maritime research support", "autonomous maritime vehicle"]\n}', 'Communication solutions': '{\n    "Product Type": ["Modem", "Radio", "Software", "Transceiver"],\n    "Technology": ["VDES technology", "acoustic modem", "reconfigurable SDR", "sonar software"],\n    "Application": ["maritime communication", "subsea communication", "underwater mapping", "navigation systems"]\n}', 'Surveilance & monitoring': '{\n    "Product Type": ["Autonomous Underwat

In [111]:
# save the dict as a excel file
df = pd.DataFrame.from_dict(tags_dict, orient="index")
df.to_excel("filter_tags.xlsx")

In [141]:
new_df_tags_filter = pd.DataFrame(columns=["category", "product_type", "technology", "application"])
for category, filters in tags_dict.items():
    # make tags into a dict
    filters_dict = ast.literal_eval(filters)
    # get the tags for each category

    
    # print("category", category, ", tag_category", filters_dict.keys(), ", tag", filters_dict.values() )
    for k in filters_dict.keys():
        if "Product Type" in k:
            product_type = filters_dict["Product Type"]
        elif "Technology" in k:
            technology = filters_dict["Technology"]
        elif "Application" in k:
            application = filters_dict["Application"]
    
    print("product_type", ', '.join(product_type), ", technology", ', '.join(technology), ", application", ', '.join(application))
    new_row = {"category": category, "product_type": ', '.join(product_type), "technology": ', '.join(technology), "application": ', '.join(application)}    
    new_df_tags_filter.loc[len(new_df_tags_filter)] = new_row # only use with a RangeIndex!
    
    # add the category in columns category, tags in columns product_type if tag_category is product_type etc.
    # new_row = {"category": category, "product_type": 
    #             "product_type": tag if tag_category == "product_type" else None,
    #             "technology": tag if tag_category == "technology" else None,
    #             "application": tag if tag_category == "application" else None}
    # new_df_tags_filter.loc[len(new_df_tags_filter)] = new_row # only use with a RangeIndex!


            
print(new_df_tags_filter)
new_df_tags_filter.to_excel("filter_tags.xlsx")

product_type Autonomous Underwater Vehicle, Diver Detection Sonar , technology advanced navigation system, 3D sonar technology, multi-sensor payload, adaptive autonomy , application underwater exploration, maritime surveillance, harbor defense, marine survey
product_type Uncrewed Surface Vehicle , technology remote vessel operation, customizable payload, USV hydroacoustic applications , application long endurance, maritime research support, autonomous maritime vehicle
product_type Modem, Radio, Software, Transceiver , technology VDES technology, acoustic modem, reconfigurable SDR, sonar software , application maritime communication, subsea communication, underwater mapping, navigation systems
product_type Autonomous Underwater Vehicle, Acoustic Control System, Leak Detection System, Diver Detection Sonar , technology 3D sonar technology, acoustic telemetry link, AIS transponder, autonomous navigation , application underwater surveillance, remote sensing, BOP operation, marine survey
pr

In [81]:
new_df = pd.read_excel("filter_tags.xlsx")

new_df.to_csv("filter_tags.csv", index=False)

# #save dataframe as json file
# test_df = new_df.apply(lambda x: ', '.join(sorted([i.strip() for i in x.split(',')])) if isinstance(x, str) else x)
# print(test_df["technology"][1])



In [115]:
def sort_tags(tags):
    if isinstance(tags, str):
        # Split the string into a list of tags
        original_tags_list = tags.split(',')
        # Strip leading/trailing spaces and convert all tags to lowercase
        lowercase_tags_list = [tag.strip().lower() for tag in original_tags_list]
        # Create a mapping from lowercase tags to original tags
        tag_mapping = list(zip(lowercase_tags_list, original_tags_list))
        # Sort the list of lowercase tags
        sorted_tags_list = sorted(tag_mapping)
        # Convert the sorted lowercase tags back to the original case
        tags_list = [tag[1].strip() for tag in sorted_tags_list]
        # Capitalize the first letter of each word in a tag if the tag does not contain any uppercase characters
        tags_list = [tag if any(char.isupper() for char in tag) else tag.title() for tag in tags_list]
        # Join the sorted tags back into a string
        tags = ', '.join(tags_list)
    return tags

# Apply the function to each cell in the DataFrame
new_df = new_df.map(sort_tags)

new_df.to_csv("filter_tags.csv", index=False)
new_df.to_excel("filter_tags.xlsx", index=False)

In [ ]:
# #save tags_dict as a colum with coresponding category
# # df_tag_tech_test = pd.read_csv("./data/df_tags_technology.csv")
# df_tags_tech["tags_dict"] = ""
# for index, row in df_tags_tech.iterrows():
#     print(row)
#     category = row["Product category"]
#     print(category)
#     tags_dict_category = tags_dict.get(category)
#     print(tags_dict_category)
#     df_tags_tech.loc[index, "tags_dict"] = tags_dict_category


# df_tags_tech.to_csv("./data/tags_27_11.csv", index=False)

In [33]:
# Create a new column to store the tags for each product
df_tech_new_tags = pd.read_csv("./data/tags_27_11.csv", usecols=["Product_Name", "Product category", "url", "image_url", "is_software"])
tags_df = pd.read_csv("filter_tags.csv")

# Set 'category' as the index of tags_df for easier lookup
tags_df.set_index('category', inplace=True)

# Iterate over the rows of the dataframe
for index, row in df_tech_new_tags.iterrows():
    print(index)
    product_category = str(row["Product category"])  # Convert to string
    # Get the tags for the product category
    if product_category in tags_df.index:
        category_tags = tags_df.loc[product_category].to_dict()  # Use a different variable
        print(row["Product_Name"])
        url = row["url"]
        website_text = url_text_dict.get(url, "")

        print(category_tags)
        # choose 8 tags from the tags list for each product
        messages = [
            {
                "role": "system",
                "content": "You are a helpful assistant that generates relevant tags for products. Your responses should be a comma-separated list of tags. For example tag1, tag2, tag3, etc..."
            },
            {
                "role": "user",
                "content": f"For the product named '{row['Product_Name']}' in the category ' {row['Product category']} ' with the following description: '{website_text}', please select up to 2 tags from each top-level category from the provided list: {category_tags}. The top-level categories include 'Product Type', 'Technology', and 'Application'. The maximum total number of tags is 6, but you are not required to use all 6. Your response should be a comma-separated list of tags. Please note: avoid using 'tag' as a tag, and do not use the top-level category names as tags."
            }
        ]
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=messages
            # response_format={ "type": "json_object" }
        )
        tags = response.choices[0].message.content.strip()
        df_tech_new_tags.loc[index, "product_tags"] = tags
        # print(row["Product_Name"])
        print(tags)
        print("______________")
    else:
        print(f"Category '{product_category}' not found in tags_df")

# save the dataframe as csv
df_tech_new_tags.to_csv("./data/tags_27_11.csv", index=False)








0
K-Sync Synchronization unit
{'product type': 'Transponder, Positioning system, Navigation system, Modem', 'technology': 'Inertial Measurement Unit, acoustic telemetry, Hydroacoustic Positioning', 'application': 'dynamic positioning reference, underwater communication, ROV integration, subsea survey'}


/var/folders/3c/7nbqh9x54rsgdpqc1dpcxh7w0000gn/T/ipykernel_80870/470496401.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Positioning system, Navigation system, Hydroacoustic Positioning, underwater communication, subsea survey' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_tech_new_tags.loc[index, "product_tags"] = tags


Positioning system, Navigation system, Hydroacoustic Positioning, underwater communication, subsea survey
______________
1
cNODE Midi - Transponder
{'product type': 'Transponder, Positioning system, Navigation system, Modem', 'technology': 'Inertial Measurement Unit, acoustic telemetry, Hydroacoustic Positioning', 'application': 'dynamic positioning reference, underwater communication, ROV integration, subsea survey'}
Transponder, Navigation system, acoustic telemetry, Hydroacoustic Positioning, underwater communication, ROV integration
______________
2
cNODE - Transponder, Embedable
{'product type': 'Transponder, Positioning system, Navigation system, Modem', 'technology': 'Inertial Measurement Unit, acoustic telemetry, Hydroacoustic Positioning', 'application': 'dynamic positioning reference, underwater communication, ROV integration, subsea survey'}
Transponder, Positioning system, acoustic telemetry, Hydroacoustic Positioning, underwater communication, ROV integration
_____________

In [ ]:
df_tech_new_tags.to_csv("./data/tags_27_11.csv", index=False)

In [ ]:
# for index, row in df_tech_new_tags.iterrows():
#     print(index)
#     product_category = row["Product category"]
#     # Get the tags for the product category
#     if product_category in tags_dict:
#         tags = tags_dict[product_category]
#         print(tags)

#         # Remove "High-Level Tag" from the tags
#         tags = [tag for tag in tags if tag != "High-Level Tag"]

#         # choose 8 tags from the tags list for each product
#         prompt = "Given the list of tags: {} and the product category :{} and product: {}, please select 8 tags from the list of tags, that can be used as filters to best find this product on a website. Each secondlevel tag should correspond to minimun one product each. Your response should only be a comma-separated list of exactly 8 tags from the given list.".format(
#             tags,  row["Product category"], row["Product_Name"]
#         )
#         tags = general_gpt(prompt)
#         df_tech_new_tags.loc[index, "tags"] = tags
#     else:
#         print(f"Category '{product_category}' not found in tags_dict")

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# def extract_image_url(url):
#     print(url)
#     if url == "https://www.kongsberg.comn":
#         return "https://www.feed-image-editor.com/sites/default/files/perm/wysiwyg/image_not_available.png"
#     if pd.isna(url) or 'http' not in url:
#         return "https://www.feed-image-editor.com/sites/default/files/perm/wysiwyg/image_not_available.png"
#     # Get the HTML of the page
#     response = requests.get(url)

#     # If the response status code is 404, return the dummy image URL
#     if response.status_code == 404:
#         return "https://www.feed-image-editor.com/sites/default/files/perm/wysiwyg/image_not_available.png"

#     html = response.text

#     # Parse the HTML with BeautifulSoup
#     soup = BeautifulSoup(html, 'html.parser')

#     # Find the image
#     img = soup.find('img')

#     # If the image was found, return its source URL
#     if img is not None:
#         img_url = "https://www.kongsberg.com"+img['src']
#         return img_url
#     else:
#         return "https://www.feed-image-editor.com/sites/default/files/perm/wysiwyg/image_not_available.png"
# # iterate over the rows of the dataframe, and find the image URL for each url and save it in a new column
# df_tech_new_tags = pd.read_csv("./data/df_tags_use_app_22_11.csv")
# df_tech_new_tags["image_url"] = df_tech_new_tags["url"].apply(extract_image_url)

# # save the dataframe as csv
# df_tech_new_tags.to_csv("./data/df_tags_use_app_22_11.csv", index=False)

In [34]:
df_tech_new_tags.to_excel("./data/tags_27_11.xlsx", index=False)
df_tech_new_tags.to_csv("./data/tags_27_11.csv", index=False)


In [35]:
import ast
def extract_tags(tags_str):
    tags_obj = ast.literal_eval(tags_str)
    if isinstance(tags_obj, dict):
        tags_values = tags_obj.values()
    elif isinstance(tags_obj, tuple):
        tags_values = tags_obj
    else:
        raise ValueError(f"Unexpected type {type(tags_obj)}: {tags_obj}")
    
    tags_list = []
    for value in tags_values:
        if isinstance(value, list):
            tags_list.append(', '.join(map(str.strip, value)))
        elif isinstance(value, dict):
            tags_list.append(str(value))
        else:
            tags_list.append(value.strip())
    return ', '.join(tags_list)

In [36]:
df_with_many_tags = pd.read_csv("data/tags_27_11.csv")
print(len(df_with_many_tags))
# df_with_many_tags['product_tags'] = df_with_many_tags['product_tags'].apply(extract_tags)
# combine all the tags with same product_name into one row. Let the name be the same as the first row, but for tags, combine them into two lists of tags. For the category, combine them into a list of categories. For website_url, image_url, choose the first one. 
df_with_many_tags = df_with_many_tags.groupby('Product_Name').agg({
    'product_tags': lambda x: ', '.join(set(map(lambda y: str(y).strip(), x))),
    'Product category': lambda x: ', '.join(map(lambda y: str(y).strip(), x)),
    'url': 'first',
    'image_url': 'first'
    
}).reset_index()
print(len(df_with_many_tags))

# save to excel
df_with_many_tags.to_excel("data/df_with_many_tags_27_11.xlsx", index=False)
df_with_many_tags.to_csv("data/df_with_many_tags_27_11.csv", index=False)

298
211


In [ ]:
# # Iterate over the rows of the dataframe
# for index, row in df_tech_new_tags.iterrows():
#     print(index)
#     # Check if the "tags" column starts with "product"
#     # Check if the "tags" column starts with "product" or "Product"
#     if "Product Type" in str(row["tags"]):
#     # if str(row["tags"]).startswith("product") or str(row["tags"]).startswith("Product") or str(row["tags"]).startswith("Product Type"):
#         print("found occurence")

#         product_category = row["Product category"]
#         # Get the tags for the product category
#         if product_category in tags_dict:
#             print("found category")
#             tags_dict = tags_dict[product_category]

#             # choose 5 tags from the tags list for each product
#             prompt = "Given the list of tags: {} and the product category :{} and product: {}, please select 5 tags from the list of tags, that can be used as filters to best find this product on a website. Each secondlevel tag should correspond to minimun one product each. Your response should only be a comma-separated list of exactly 5 tags from the given list.".format(
#                 tags_dict,  row["Product category"], row["Product_Name"]
#             )
#             tags = general_gpt(prompt)
#             df_tech_new_tags.loc[index, "tags"] = tags
#             print(tags)
#         else:
#             print(f"Category '{product_category}' not found in tags_dict")

# # save the dataframe as excel file
# df_tech_new_tags.to_excel("./data/df_tags_use_app_15_11.xlsx", index=False)

In [ ]:
# import ast
# df_tech_new_tags = pd.read_csv("./data/df_tags_use_app_15_11.csv")

# # Iterate over the rows of the dataframe
# for index, row in df_tech_new_tags.iterrows():
#     print(index)

#     product_category = row["Product category"]
#     tags_dict_str = row["tags_dict"]

#     if pd.notna(tags_dict_str):
#         # Convert the string representation of dictionary to a dictionary
#         tags_dict = ast.literal_eval(tags_dict_str)

#         # Check if "High-Level Tag" is in the values of tags_dict
#         if "High-Level Tag" in tags_dict.values():
#             # Repeat the process
#             prompt = "Given the list of tags: {} and the product category :{} and product: {}, please select 5 tags from the list of tags, that can be used as filters to best find this product on a website. Each secondlevel tag should correspond to minimun one product each. Your response should only be a comma-separated list of exactly 5 tags from the given list.".format(
#                 tags_dict,  row["Product category"], row["Product_Name"]
#             )
#             tags = general_gpt(prompt)
#             df_tech_new_tags.loc[index, "tags"] = tags
#             print(tags)
#         else:
#             print(f"'High-Level Tag' not found in tags_dict values")
#     else:
#         print("tags_dict_str is NaN")

# # save the dataframe as excel file
# df_tech_new_tags.to_excel("./data/df_tags_use_app_15_11.xlsx", index=False)

In [ ]:
# df_tech_new_tags = pd.read_csv("./data/tags_27_11.csv")

In [ ]:
# import re
# ant = 0
# for index, row in df_tech_new_tags.iterrows():
#     print(index)
#     if not pd.isna(row["tags"]):
#         # ant_tags = row["tags"].split(",")
#         ant_tags = re.split(',|:', row["tags"])
#         print(ant_tags)
#         if len(ant_tags) > 12:
#         # if "'Product Type'" in ant_tags or  "Product Type" in ant_tags or " Product Type" in ant_tags or "Product Type " in ant_tags or " Application" in ant_tags or " Technology" in ant_tags:
                
#             ant += 1
#             print(index)
#             print(row["tags"])
#             print(len(ant_tags))
#             print("found more than 10 tags")
#             print("------------------")
#             # print(row["tags_dict"])
#             print("updating tags")
#             # prompt = "Use this list of tags {}. And for this product: {}, please select 8 tags from the list of tags that can be used as filters to best find this product on a website. Your response should only be a comma-separated list of exactly 8 tags from the given list. Do not use the word 'tag' as a tag. Dont use Product Type, Technology or Application as tags either.".format(            
#             # tags_dict,  row["Product category"], row["Product_Name"]
#             # )
#             # tags = general_gpt(prompt)
#             # print("New tags: ", tags)
#             # df_tech_new_tags.loc[index, "tags"] = tags
#             print("------------------")
#             # update the tags column with only 8 tags


# print("ant, ",ant)

# # print(df_tech_new_tags["tags"])

In [ ]:
# df_tech_new_tags.to_csv("./data/df_tags_use_app_15_11.csv", index=False)
# df_tech_new_tags.to_excel("./data/df_tags_use_app_15_11.xlsx", index=False)

In [ ]:
# # Read the dataframe from the CSV file
# df_tags_tech = pd.read_csv("./data/tags_gpt4.csv")

# # Group the dataframe by "Product category" and aggregate the "tags" column
# grouped_tags = df_tags_tech.groupby("Product category")["tags"].agg(list)

# # Create a dictionary to store the tags for each category
# all_tags_by_category = {}

# for category, tags_list in grouped_tags.items():
#     # Flatten the list of tags
#     tags = [tag.strip("' ").strip() for tags in tags_list if isinstance(tags, str) for tag in tags.split(",")]
#     # Remove duplicates
#     tags = list(set(tags))
#     # Add the tags to the dictionary
#     all_tags_by_category[category] = ",".join(tags)

# print(all_tags_by_category)

In [ ]:
# import json
# df_tags_tech = pd.read_csv("./data/tags_gpt4.csv")
# # Create a set of all product tags
# product_tags = set()
# for category, group in df_tags_tech.groupby("Product category"):
#     category_tags = [tag.strip("' ").strip() for tag in all_tags_by_category[category].split(',')]
#     product_tags.update(category_tags)

# non_matching_tags = []

# # Iterate over each tag in dict_tags
# for index, row in df_tags_tech.iterrows():
#     tags_dict_str = row["tags_dict"]
#     if pd.notna(tags_dict_str):
#         tags_dict_str = tags_dict_str.replace("'", '"')
#         print(tags_dict_str)  # Add this line
#         try:
#             tags_dict = json.loads(tags_dict_str)
#         except json.JSONDecodeError as e:
#             print(f"Error decoding JSON for row {index}: {e}")
#             continue

#         for tag in tags_dict.values():
#             for t in tag:
#                 # Strip leading/trailing whitespace and quotes from t
#                 t = t.strip("' ").strip()
#                 if t not in product_tags:
#                     non_matching_tags.append(t)

# # Print non-matching tags
# if non_matching_tags:
#     print("The following tags do not have any corresponding tags in their category:")
#     for tag in set(non_matching_tags):
#         print(tag)


In [ ]:
# # Load the data
# df = pd.read_csv("./data/good_data/tags_gpt4.csv")

# # print len of tags that are empty or null
# print(len(df[df['tags'].isnull()]))
# #remove if tags is null
# df = df[df['tags'].notnull()]
# print(len(df[df['tags'].apply(lambda x: len(x) == 0)]))

# # Print unique values in the 'tags' column
# # print(df.columns)
# # print(df['tags'].unique())
# # # Print the 'tags' column
# # print(df['tags'])

# # Convert the 'tags' column to a dictionary
# df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

# # Extract the values of the key 'tags' and split them into a list for each row
# df['tag_values'] = df['tags'].apply(lambda x: x['tags'].split(', ') if isinstance(x, dict) and 'tags' in x and isinstance(x['tags'], str) else [])

# # Print the 'tag_values' column
# # print(df['tag_values'])

# # print all the empty list of tags
# print(len(df[df['tag_values'].apply(lambda x: len(x) == 0)]))